# **Importing required libraries.**

In [1]:
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
from pandas.core.frame import DataFrame
import tensorflow as tf
from collections import OrderedDict
from tensorflow.python.keras.layers.recurrent import RNN
import os
import glob
import cv2
from matplotlib import pyplot as plt
from collections import defaultdict
from numpy import savez_compressed
from sklearn import linear_model, datasets, model_selection, preprocessing


# **Calling the images folder from Google drive & listing sub dircetories .**

In [2]:
path = "/content/drive/MyDrive/Images150/"
br_path = os.listdir(path)
print (len(br_path))

120


## **Looping on sub directories in order to store thier paths.**

In [3]:
speci_paths_list =[]
for item in br_path:
    speci_path = path+item
    speci_paths_list.append(speci_path)
    print(speci_path)
print(len(speci_paths_list))

/content/drive/MyDrive/Images150/Yorkshire_terrier
/content/drive/MyDrive/Images150/toy_poodle
/content/drive/MyDrive/Images150/West_Highland_white_terrier
/content/drive/MyDrive/Images150/Welsh_springer_spaniel
/content/drive/MyDrive/Images150/Walker_hound
/content/drive/MyDrive/Images150/wire-haired_fox_terrier
/content/drive/MyDrive/Images150/whippet
/content/drive/MyDrive/Images150/vizsla
/content/drive/MyDrive/Images150/Weimaraner
/content/drive/MyDrive/Images150/toy_terrier
/content/drive/MyDrive/Images150/silky_terrier
/content/drive/MyDrive/Images150/Sussex_spaniel
/content/drive/MyDrive/Images150/standard_poodle
/content/drive/MyDrive/Images150/Tibetan_terrier
/content/drive/MyDrive/Images150/Shih-Tzu
/content/drive/MyDrive/Images150/Siberian_husky
/content/drive/MyDrive/Images150/Staffordshire_bullterrier
/content/drive/MyDrive/Images150/standard_schnauzer
/content/drive/MyDrive/Images150/soft-coated_wheaten_terrier
/content/drive/MyDrive/Images150/Tibetan_mastiff
/content/dr

**Reading each sub directory images & store each image path with its kind from folder name.**

In [4]:
i=0
list_of_speci = []
list_of_images=[]
images_pathes=[]
while i < len(speci_paths_list):
    mini_path = speci_paths_list[i]+"//"+ "*.*"
    image_list = glob.glob(mini_path)
    for image_path in image_list:
        images_pathes.append(image_path)
        list_of_speci.append(br_path[i])
    print("finished reading",br_path[i],"images")
    print(br_path[i],mini_path,i)
    print("====================================================================================================================================================================================================")
    i=i+1
print(len(images_pathes))

finished reading Yorkshire_terrier images
Yorkshire_terrier /content/drive/MyDrive/Images150/Yorkshire_terrier//*.* 0
finished reading toy_poodle images
toy_poodle /content/drive/MyDrive/Images150/toy_poodle//*.* 1
finished reading West_Highland_white_terrier images
West_Highland_white_terrier /content/drive/MyDrive/Images150/West_Highland_white_terrier//*.* 2
finished reading Welsh_springer_spaniel images
Welsh_springer_spaniel /content/drive/MyDrive/Images150/Welsh_springer_spaniel//*.* 3
finished reading Walker_hound images
Walker_hound /content/drive/MyDrive/Images150/Walker_hound//*.* 4
finished reading wire-haired_fox_terrier images
wire-haired_fox_terrier /content/drive/MyDrive/Images150/wire-haired_fox_terrier//*.* 5
finished reading whippet images
whippet /content/drive/MyDrive/Images150/whippet//*.* 6
finished reading vizsla images
vizsla /content/drive/MyDrive/Images150/vizsla//*.* 7
finished reading Weimaraner images
Weimaraner /content/drive/MyDrive/Images150/Weimaraner//*

# Converting images list & labels into numpy array for further proccessing.

In [5]:
images_paths = np.array(images_pathes)
images_labels = np.array(list_of_speci)

# **Creating the data frame.**

In [6]:
raw_frame= pd.DataFrame(list(zip(images_pathes,images_labels)),columns =['images', 'labels'])

# **Defining function that will be used to read images pathes & coverting them to images.**

In [7]:
def get_image_tensor_from_path(image_path, label):

    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.resize(image, (299,299))
    return image, label


# **Defining function will be used to create datasets that will be used in the training & testing phase.**

In [8]:
def dataset_creator(x, y, batch_size = 32):

  data = tf.data.Dataset.from_tensor_slices((x,y))
  data = data.shuffle(2000)
  data = data.map(get_image_tensor_from_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  data = data.batch(batch_size)
  data = data.prefetch(tf.data.experimental.AUTOTUNE)
  return data


# **Converting labels into one hot encoding  replacing the old label column with onehot values.**

In [9]:

x = raw_frame.images
y = raw_frame.labels
x = np.array(x)
y = np.array(y)
speci_index = dict(enumerate(list(OrderedDict.fromkeys(y.tolist()))))
print(speci_index)
speci_index_rev = {value: key for (key, value) in speci_index.items()}
print (speci_index_rev)
data_t = pd.DataFrame(list(zip(x,y)),columns =["images", "labels"])
data_t = data_t.replace({"labels":speci_index_rev})
print(data_t.head(10))

{0: 'Yorkshire_terrier', 1: 'toy_poodle', 2: 'West_Highland_white_terrier', 3: 'Welsh_springer_spaniel', 4: 'Walker_hound', 5: 'wire-haired_fox_terrier', 6: 'whippet', 7: 'vizsla', 8: 'Weimaraner', 9: 'toy_terrier', 10: 'silky_terrier', 11: 'Sussex_spaniel', 12: 'standard_poodle', 13: 'Tibetan_terrier', 14: 'Shih-Tzu', 15: 'Siberian_husky', 16: 'Staffordshire_bullterrier', 17: 'standard_schnauzer', 18: 'soft-coated_wheaten_terrier', 19: 'Tibetan_mastiff', 20: 'schipperke', 21: 'Shetland_sheepdog', 22: 'Sealyham_terrier', 23: 'Scottish_deerhound', 24: 'Saint_Bernard', 25: 'Saluki', 26: 'Rottweiler', 27: 'Scotch_terrier', 28: 'Samoyed', 29: 'Rhodesian_ridgeback', 30: 'Pembroke', 31: 'Old_English_sheepdog', 32: 'Pekinese', 33: 'Norwegian_elkhound', 34: 'pug', 35: 'Pomeranian', 36: 'otterhound', 37: 'papillon', 38: 'Norwich_terrier', 39: 'redbone', 40: 'Newfoundland', 41: 'malinois', 42: 'miniature_poodle', 43: 'Mexican_hairless', 44: 'Maltese_dog', 45: 'Norfolk_terrier', 46: 'Lhasa', 47: 

In [11]:
x_cab = data_t.images
y_cab = data_t.labels
y_cab = tf.one_hot(y_cab,depth=120)


# **Setting the value of random seed.**

In [12]:
tf.random.set_seed(42)

# **Creating the train & test X,Y & spliting the data into 0.8 training , 0.2 testing.**

In [13]:
x_cab = np.array(x_cab)
y_cab = np.array(y_cab)
x_train, x_test, y_train, y_test= model_selection.train_test_split(x_cab, y_cab, test_size = 0.2, random_state= 42,stratify=y)

# **Creating the train & test datasets.**

In [14]:
train_data = dataset_creator(x_train,y_train)
test_data = dataset_creator(x_test,y_test)

# **Creating our CNN model.**

In [15]:
model = tf.keras.models.Sequential([tf.keras.layers.Input([500,500,3])
,tf.keras.layers.Conv2D(32, (5, 5), activation='relu',padding='same')
,tf.keras.layers.MaxPooling2D((2, 2))
,tf.keras.layers.Conv2D(64, (5, 5), activation='relu',padding='same')
,tf.keras.layers.MaxPooling2D((2, 2))
,tf.keras.layers.Flatten()
,tf.keras.layers.Dense(512, activation='relu')
,tf.keras.layers.Dense(512, activation='relu')
,tf.keras.layers.Dense(256, activation='relu')
,tf.keras.layers.Dense(128, activation='relu')
,tf.keras.layers.Dense(120,activation=tf.nn.softmax)
]
)

# **Printing the model summary.**

In [16]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 500, 500, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 250, 250, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 250, 250, 64)      51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 125, 125, 64)      0         
_________________________________________________________________
flatten (Flatten)            (None, 1000000)           0         
_________________________________________________________________
dense (Dense)                (None, 512)               512000512 
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2

# **Compiling our model fitting on training data & we as we see the loss is to much so we will test again using Imagnet pretrained model.**

In [ ]:
model.compile(optimizer='adam',loss=tf.losses.CategoricalCrossentropy(),metrics=['acc'])
,tf.keras.layers.MaxPooling2D((2, 2))
history=model.fit(train_data,epochs=100,validation_data=test_data)

Epoch 1/100
450/450 [==============================] - 3446s 8s/step - loss: 5.7157 - acc: 0.0075 - val_loss: 4.7875 - val_acc: 0.0083
Epoch 2/100
450/450 [==============================] - 156s 347ms/step - loss: 4.7880 - acc: 0.0052 - val_loss: 4.7875 - val_acc: 0.0083
Epoch 3/100
450/450 [==============================] - 157s 348ms/step - loss: 4.7881 - acc: 0.0046 - val_loss: 4.7875 - val_acc: 0.0083
Epoch 4/100
450/450 [==============================] - 158s 350ms/step - loss: 4.7881 - acc: 0.0053 - val_loss: 4.7875 - val_acc: 0.0083
Epoch 5/100
450/450 [==============================] - 157s 348ms/step - loss: 4.7881 - acc: 0.0059 - val_loss: 4.7875 - val_acc: 0.0083
Epoch 6/100
450/450 [==============================] - 157s 348ms/step - loss: 4.7882 - acc: 0.0058 - val_loss: 4.7875 - val_acc: 0.0083
Epoch 7/100
450/450 [==============================] - 157s 349ms/step - loss: 4.7882 - acc: 0.0060 - val_loss: 4.7875 - val_acc: 0.0083
Epoch 8/100
450/450 [======================

# **Imagenet pretrained model modification in order to fit on our data.**

In [16]:
base_model = tf.keras.applications.InceptionResNetV2(
    weights='imagenet',
    input_shape=(299, 299, 3),
    include_top=False)

# **Creating checkpoint in order to save model.**

In [17]:
checkpoint_filepath = '/content/ahmed_l.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_acc',
    mode='max',
    save_best_only=True,)

In [18]:
base_model.trainable = False

In [19]:
base_model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_203 (Conv2D)             (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_203 (BatchN (None, 149, 149, 32) 96          conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 149, 149, 32) 0           batch_normalization_203[0][0]    
________________________________________________________________________________

In [21]:
inputs = tf.keras.Input(shape=(299, 299, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.Flatten()(x)
x=tf.keras.layers.Dense(128,activation=tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(120,activation=tf.nn.softmax)(x)
model_cnn = tf.keras.Model(inputs, outputs)

In [22]:
model_cnn.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
inception_resnet_v2 (Functio (None, 8, 8, 1536)        54336736  
_________________________________________________________________
flatten_1 (Flatten)          (None, 98304)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               12583040  
_________________________________________________________________
dense_3 (Dense)              (None, 120)               15480     
Total params: 66,935,256
Trainable params: 12,598,520
Non-trainable params: 54,336,736
_________________________________________________________________


# **Fitting the pretrained model  . **

In [ ]:
model_cnn.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['acc'])
history=model_cnn.fit(train_data, epochs=100,validation_data=test_data,callbacks=model_checkpoint_callback)

Epoch 1/100
450/450 [==============================] - 203s 425ms/step - loss: 2.5935 - acc: 0.5339 - val_loss: 0.7943 - val_acc: 0.8083
Epoch 2/100
450/450 [==============================] - 187s 414ms/step - loss: 0.6170 - acc: 0.8486 - val_loss: 0.8061 - val_acc: 0.8236
Epoch 3/100
450/450 [==============================] - 187s 415ms/step - loss: 0.4308 - acc: 0.8881 - val_loss: 0.7970 - val_acc: 0.8389
Epoch 4/100
450/450 [==============================] - 187s 415ms/step - loss: 0.3149 - acc: 0.9114 - val_loss: 0.8782 - val_acc: 0.8336
Epoch 5/100
450/450 [==============================] - 186s 414ms/step - loss: 0.2851 - acc: 0.9243 - val_loss: 0.9190 - val_acc: 0.8517
Epoch 6/100
450/450 [==============================] - 187s 414ms/step - loss: 0.2150 - acc: 0.9424 - val_loss: 0.9961 - val_acc: 0.8344
Epoch 7/100
450/450 [==============================] - 187s 415ms/step - loss: 0.2711 - acc: 0.9314 - val_loss: 1.1239 - val_acc: 0.8442
Epoch 8/100
450/450 [====================